In [4]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest ,mutual_info_classif, mutual_info_regression, f_classif, chi2
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

In [5]:
data = pd.read_csv("data/train_values.csv")
data.head()

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
0,0,3,1,1,1,70,3,18,37,246,...,24.0,6203.0,44.230,60588.0,50.933,716.0,2642.0,4536,False,1
1,1,1,1,3,1,178,3,369,52,299,...,57.0,5774.0,15.905,54821.0,100.000,1622.0,2108.0,2458,False,0
2,2,2,1,3,1,163,3,16,10,306,...,67.0,6094.0,61.270,67719.0,100.000,760.0,1048.0,5710,False,1
3,3,1,1,1,1,155,1,305,47,180,...,105.0,6667.0,6.246,78439.0,100.000,2025.0,2299.0,5888,True,1
4,4,1,1,1,1,305,3,24,37,20,...,71.0,6732.0,100.000,63075.0,82.200,1464.0,1847.0,289,False,1


In [63]:
pd.DataFrame(data.groupby(["preapproval","accepted"])["accepted"].count())

accepted
preapproval accepted          
1           0            20226
            1             8522
2           0            20677
            1            39466
3           0           208983
            1           202126

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 23 columns):
row_id                            500000 non-null int64
loan_type                         500000 non-null int64
property_type                     500000 non-null int64
loan_purpose                      500000 non-null int64
occupancy                         500000 non-null int64
loan_amount                       500000 non-null int64
preapproval                       500000 non-null int64
msa_md                            500000 non-null int64
state_code                        500000 non-null int64
county_code                       500000 non-null int64
applicant_ethnicity               500000 non-null int64
applicant_race                    500000 non-null int64
applicant_sex                     500000 non-null int64
applicant_income                  460052 non-null float64
population                        477535 non-null float64
minority_population_pct           477534 non-

In [65]:
data.describe()

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,accepted
count,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,...,500000.000000,460052.000000,477535.000000,477534.000000,477560.000000,477486.000000,477435.000000,477470.000000,500000.000000,500000.000000
mean,249999.500000,1.366276,1.047650,2.066810,1.109590,221.753158,2.764722,181.606972,23.726924,144.542062,...,1.462374,102.389521,5416.833956,31.617310,69235.603298,91.832624,1427.718282,1886.147065,3720.121344,0.500228
std,144337.711634,0.690555,0.231404,0.948371,0.326092,590.641648,0.543061,138.464169,15.982768,100.243612,...,0.677685,153.534496,2728.144999,26.333938,14810.058791,14.210924,737.559511,914.123744,1838.313175,0.500000
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,...,1.000000,1.000000,14.000000,0.534000,17858.000000,3.981000,4.000000,1.000000,0.000000,0.000000
25%,124999.750000,1.000000,1.000000,1.000000,1.000000,93.000000,3.000000,25.000000,6.000000,57.000000,...,1.000000,47.000000,3744.000000,10.700000,59731.000000,88.067250,944.000000,1301.000000,2442.000000,0.000000
50%,249999.500000,1.000000,1.000000,2.000000,1.000000,162.000000,3.000000,192.000000,26.000000,131.000000,...,1.000000,74.000000,4975.000000,22.901000,67526.000000,100.000000,1327.000000,1753.000000,3731.000000,1.000000
75%,374999.250000,2.000000,1.000000,3.000000,1.000000,266.000000,3.000000,314.000000,37.000000,246.000000,...,2.000000,117.000000,6467.000000,46.020000,75351.000000,100.000000,1780.000000,2309.000000,5436.000000,1.000000
max,499999.000000,4.000000,3.000000,3.000000,3.000000,100878.000000,3.000000,408.000000,52.000000,324.000000,...,4.000000,10139.000000,37097.000000,100.000000,125248.000000,100.000000,8771.000000,13623.000000,6508.000000,1.000000


In [6]:
data = data.rename(columns = {'number_of_owner-occupied_units':'number_of_owner_occupied_units'})
data["co_applicant"]=(data['co_applicant']).astype(int)
# mark zero values as missing or NaN
data[data.columns] = data[data.columns].replace(-1, np.NaN)
data[data.columns] = data[data.columns].replace("", np.NaN)
# drop rows with missing values
data.dropna(inplace=True)
data = data.drop("row_id",1)

In [67]:
data.shape

(387163, 22)

In [7]:
#drop rows with high income 
values = data["applicant_income"]
remove = values[values > 250 ].values

In [8]:
data["applicant_income"] = data["applicant_income"].replace(to_replace= remove, value=np.NaN)
data["applicant_income"]
data.dropna(inplace=True)

In [70]:
#data.to_csv("clean_data.csv")

In [71]:
data.shape

(366601, 22)

In [9]:
data_num1 = data[["loan_amount", "applicant_income", "population", "minority_population_pct","ffiecmedian_family_income", "tract_to_msa_md_income_pct","number_of_owner_occupied_units","number_of_1_to_4_family_units","accepted"]]
data_num1.head()

,loan_amount,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner_occupied_units,number_of_1_to_4_family_units,accepted
0,70,24.0,6203.0,44.230,60588.0,50.933,716.0,2642.0,1
1,178,57.0,5774.0,15.905,54821.0,100.000,1622.0,2108.0,0
2,163,67.0,6094.0,61.270,67719.0,100.000,760.0,1048.0,1
3,155,105.0,6667.0,6.246,78439.0,100.000,2025.0,2299.0,1
4,305,71.0,6732.0,100.000,63075.0,82.200,1464.0,1847.0,1


In [10]:
data_cat = data.drop(data_num1.columns,1)
data_cat["accepted"] = data["accepted"]
data_cat.head()

,loan_type,property_type,loan_purpose,occupancy,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,lender,co_applicant,accepted
0,3,1,1,1,3,18.0,37.0,246.0,2,5,1,4536,0,1
1,1,1,3,1,3,369.0,52.0,299.0,1,5,1,2458,0,0
2,2,1,3,1,3,16.0,10.0,306.0,2,5,1,5710,0,1
3,1,1,1,1,1,305.0,47.0,180.0,2,5,1,5888,1,1
4,1,1,1,1,3,24.0,37.0,20.0,2,3,2,289,0,1


In [74]:
#print the counts for categorical columns 
for col in data_cat.columns:
    print('\nColumn Name:', col,)
    print(data_cat[col].value_counts())


Column Name: loan_type
1    279995
2     57243
3     25226
4      4137
Name: loan_type, dtype: int64

Column Name: property_type
1    356238
2     10363
Name: property_type, dtype: int64

Column Name: loan_purpose
3    176676
1    151827
2     38098
Name: loan_purpose, dtype: int64

Column Name: occupancy
1    335408
2     31046
3       147
Name: occupancy, dtype: int64

Column Name: preapproval
3    306116
2     50012
1     10473
Name: preapproval, dtype: int64

Column Name: msa_md
24.0     10552
305.0     8859
101.0     8614
314.0     8597
333.0     8249
         ...  
70.0        82
375.0       81
152.0       65
304.0       61
174.0       54
Name: msa_md, Length: 408, dtype: int64

Column Name: state_code
37.0    50588
6.0     30641
2.0     27524
22.0    13938
47.0    13906
25.0    12021
16.0    11956
48.0    11928
10.0    11243
32.0    11208
26.0    11165
4.0     10259
41.0    10022
28.0     9940
3.0      9567
34.0     8480
30.0     8058
9.0      7781
14.0     7305
38.0     6468
3

In [75]:
data_num2 = data_num1.drop("accepted", axis=1)
data_num = (data_num2 - data_num2.mean()) / (data_num2.max() - data_num2.min())

In [76]:
data_num["accepted"] = data_num1["accepted"]
data_num

,loan_amount,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner_occupied_units,number_of_1_to_4_family_units,accepted
0,-0.007965,-0.246102,0.017874,0.108854,-0.095600,-0.432855,-0.083642,0.056499,1
1,-0.001207,-0.113572,0.006287,-0.176281,-0.149301,0.090437,0.019700,0.017298,0
2,-0.002146,-0.073411,0.014930,0.280388,-0.029197,0.090437,-0.078623,-0.060517,1
3,-0.002646,0.079199,0.030407,-0.273513,0.070626,0.090437,0.065668,0.031319,1
4,0.006739,-0.057347,0.032162,0.670265,-0.072441,-0.099397,0.001678,-0.001862,1
...,...,...,...,...,...,...,...,...,...
499992,0.007052,0.211730,-0.018455,-0.233750,0.027624,0.090437,0.007039,-0.005092,1
499993,0.000169,0.167553,0.107872,-0.280177,-0.053221,0.090437,0.166957,0.103996,0
499996,-0.003585,0.524983,-0.056431,-0.267061,0.143892,0.090437,-0.002998,0.005259,0
499998,-0.010155,-0.177829,-0.056701,0.658981,-0.098030,-0.306263,-0.094820,-0.042238,0


In [77]:
corr = data_num.corr()
corr.style.background_gradient(cmap='coolwarm')

,loan_amount,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner_occupied_units,number_of_1_to_4_family_units,accepted
loan_amount,1,0.525129,0.018175,-0.00256084,0.299727,0.194565,0.00320304,-0.0549395,0.114547
applicant_income,0.525129,1,0.0290233,-0.0978974,0.21937,0.211246,0.0482036,-0.0157141,0.15469
population,0.018175,0.0290233,1,0.0602903,-0.0534746,0.162192,0.862679,0.851133,0.0179476
minority_population_pct,-0.00256084,-0.0978974,0.0602903,1,-0.0419318,-0.447405,-0.237305,-0.156789,-0.110269
ffiecmedian_family_income,0.299727,0.21937,-0.0534746,-0.0419318,1,-0.0477615,-0.0431276,-0.137368,0.0543474
tract_to_msa_md_income_pct,0.194565,0.211246,0.162192,-0.447405,-0.0477615,1,0.370191,0.215134,0.102119
number_of_owner_occupied_units,0.00320304,0.0482036,0.862679,-0.237305,-0.0431276,0.370191,1,0.910196,0.0403975
number_of_1_to_4_family_units,-0.0549395,-0.0157141,0.851133,-0.156789,-0.137368,0.215134,0.910196,1,0.0118397
accepted,0.114547,0.15469,0.0179476,-0.110269,0.0543474,0.102119,0.0403975,0.0118397,1


In [78]:

def select_kbest_clf(data_frame, target, method ,k):
    """
    Selecting K-Best features for classification
    :param data_frame: A pandas dataFrame with the training data
    :param target: target variable name in DataFrame
    :param k: desired number of features from the data
    :returns feature_scores: scores for each feature in the data as 
    pandas DataFrame
    """
    feat_selector = SelectKBest(method, k=k)
    _ = feat_selector.fit(data_frame.drop(target, axis=1), data_frame[target])
    
    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = feat_selector.scores_
    feat_scores["P Value"] = feat_selector.pvalues_
    feat_scores["Support"] = feat_selector.get_support()
    feat_scores["Attribute"] = data_frame.drop(target, axis=1).columns
    
    return feat_scores 

In [79]:

select_kbest_clf(data_num, "accepted" ,method=f_classif, k=4 )

,F Score,P Value,Support,Attribute
0,4874.111506,0.000000e+00,True,loan_amount
1,8987.410524,0.000000e+00,True,applicant_income
2,118.126041,1.642945e-27,False,population
3,4512.452103,0.000000e+00,True,minority_population_pct
4,1086.007237,8.117142e-238,False,ffiecmedian_family_income
5,3863.295529,0.000000e+00,True,tract_to_msa_md_income_pct
6,599.251482,3.113149e-132,False,number_of_owner_occupied_units
7,51.396611,7.561002e-13,False,number_of_1_to_4_family_units


In [80]:
select_kbest_clf(data_cat, "accepted" ,method=chi2, k=10)

,F Score,P Value,Support,Attribute
0,52.912577,3.487295e-13,False,loan_type
1,90.538595,1.814036e-21,True,property_type
2,6968.260471,0.000000e+00,True,loan_purpose
3,14.552915,1.362777e-04,False,occupancy
4,468.998048,5.294211e-104,True,preapproval
5,358.031546,7.554614e-80,True,msa_md
6,3902.355519,0.000000e+00,True,state_code
7,425.486756,1.560296e-94,True,county_code
8,3.181273,7.448668e-02,False,applicant_ethnicity
9,99.235596,2.241812e-23,True,applicant_race


In [81]:
select_kbest_clf(data_cat, "accepted" ,method=mutual_info_classif, k=7)

,F Score,P Value,Support,Attribute
0,0.023061,None,False,loan_type
1,0.046057,None,True,property_type
2,0.052345,None,True,loan_purpose
3,0.035952,None,True,occupancy
4,0.039251,None,True,preapproval
5,0.007126,None,False,msa_md
6,0.006362,None,False,state_code
7,0.004287,None,False,county_code
8,0.031774,None,True,applicant_ethnicity
9,0.039647,None,True,applicant_race


Grid search

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [83]:
#RandomForestClassifier?

In [84]:
rfc=RandomForestClassifier(random_state=42)

In [86]:
gsc = GridSearchCV(
        estimator=rfc,
        param_grid={
            'n_estimators': [100,200,300],
            'max_features': ["auto","sqrt","log2"],
            'max_depth': [4,5,6,7,8],
            'criterion':["gini","entropy"]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [12]:
feature_col =['loan_type','property_type','loan_purpose','occupancy', 'preapproval',
              'applicant_race','loan_amount','applicant_income',
              'minority_population_pct','tract_to_msa_md_income_pct']
X = data[feature_col] #Features
y = data['accepted'] #target

In [13]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


In [20]:
np.array(y_test)

array([0, 1, 1, ..., 0, 1, 0])

In [ ]:
grid_result = gsc.fit(X_train, y_train)

In [ ]:
best_params = grid_result.best_params_

In [ ]:
best_params

In [33]:
# Create Decision Tree classifer object
clf = RandomForestClassifier()
#10-fold cross validation
scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print("Accuracy",scores.mean())

ANN

In [ ]:
# define the model
def baseline():
    model = Sequential()
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [15]:
model = baseline()

In [16]:
model.summary()

In [37]:
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val) ,epochs = 35)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction_classes = model.predict_classes(X_test)

In [ ]:
confusion_matrix(np.argmax(Y_test,1), prediction_classes)

In [ ]:
accuracy_score(np.argmax(Y_test,1), prediction_classes)

In [1]:
def plot_hist(h, xsize=6, ysize=10):

    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]
    fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True)
    
    # summarize history for Accuracy
    plt.subplot(211)
    plt.plot(h['acc'])
    plt.plot(h['val_acc'])
    plt.title('Training Performance')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['Train', 'Validation'], loc='best')
    
    plt.draw()
    plt.show()

    return

In [ ]:
plot_hist(history.history, xsize=8, ysize=12)